In [27]:
%load_ext autoreload
%autoreload 2

import FiniteVolumeGroups as fvg
from su4ops.quark import Quark,Gamma,GammaProper
from su4ops.elemental import Elemental
from su4ops.group_rep_utils import *
from su4ops.utils import *
from su4ops.constants import *

NC=3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
o2h=fvg.cubic.O2h()

In [29]:
uQ = lambda spin : Quark({'bar': False, 'flavor': 0, 'color': 0, 'spin': spin})
uQBar = lambda spin: Quark({'bar': True, 'flavor': 0, 'color': 0, 'spin': spin})

In [30]:
allSpins=[]
basis=[]
allSpinsToBasis={}
basisIdx=0
for i in range(NS**NC):
    quarks=[]
    tmp=i
    for s in range(NC):
        quarks.append(uQ(tmp%NS))
        tmp=tmp//NS
    # match order according to equation 4.66
    quarks[0], quarks[2] = quarks[2], quarks[0]
    # add the new elemental to allSpins
    allSpins.append(Elemental(1,quarks))
    # now check if it's already in the basis
    perms = []
    for p in permutations(quarks):
        perms.append(copy.deepcopy(p))

    found=False
    for b in basis:
        if b.quarks in perms:
            found=True
            foundIdx=basis.index(b)

    if not found:
        basis.append(Elemental(1,quarks))
        allSpinsToBasis[i]=basisIdx
        basisIdx+=1
    else:
        allSpinsToBasis[i]=foundIdx

#for b in basis:
    #print(b)
    
print(len(basis))
print(len(allSpinsToBasis))
print(allSpinsToBasis)

20
64
{0: 0, 1: 1, 2: 2, 3: 3, 4: 1, 5: 4, 6: 5, 7: 6, 8: 2, 9: 5, 10: 7, 11: 8, 12: 3, 13: 6, 14: 8, 15: 9, 16: 1, 17: 4, 18: 5, 19: 6, 20: 4, 21: 10, 22: 11, 23: 12, 24: 5, 25: 11, 26: 13, 27: 14, 28: 6, 29: 12, 30: 14, 31: 15, 32: 2, 33: 5, 34: 7, 35: 8, 36: 5, 37: 11, 38: 13, 39: 14, 40: 7, 41: 13, 42: 16, 43: 17, 44: 8, 45: 14, 46: 17, 47: 18, 48: 3, 49: 6, 50: 8, 51: 9, 52: 6, 53: 12, 54: 14, 55: 15, 56: 8, 57: 14, 58: 17, 59: 18, 60: 9, 61: 15, 62: 18, 63: 19}


In [31]:
for b in basis:
    print(b)

1*u_{a,0}*u_{a,0}*u_{a,0}
1*u_{a,0}*u_{a,0}*u_{a,1}
1*u_{a,0}*u_{a,0}*u_{a,2}
1*u_{a,0}*u_{a,0}*u_{a,3}
1*u_{a,0}*u_{a,1}*u_{a,1}
1*u_{a,0}*u_{a,1}*u_{a,2}
1*u_{a,0}*u_{a,1}*u_{a,3}
1*u_{a,0}*u_{a,2}*u_{a,2}
1*u_{a,0}*u_{a,2}*u_{a,3}
1*u_{a,0}*u_{a,3}*u_{a,3}
1*u_{a,1}*u_{a,1}*u_{a,1}
1*u_{a,1}*u_{a,1}*u_{a,2}
1*u_{a,1}*u_{a,1}*u_{a,3}
1*u_{a,1}*u_{a,2}*u_{a,2}
1*u_{a,1}*u_{a,2}*u_{a,3}
1*u_{a,1}*u_{a,3}*u_{a,3}
1*u_{a,2}*u_{a,2}*u_{a,2}
1*u_{a,2}*u_{a,2}*u_{a,3}
1*u_{a,2}*u_{a,3}*u_{a,3}
1*u_{a,3}*u_{a,3}*u_{a,3}


In [32]:
rep = []
for g in o2h.elements:
  rep.append(makeRepMat(g, basis, allSpinsToBasis))

In [33]:
names = [elem.identifier['name'] for elem in o2h.elements]
indices = lambda name : [i for i,x in enumerate(names) if x==name]
print(indices("C4y"))
print(indices("C4z"))
print(indices("E"))

[20, 44, 68, 92]
[22, 46, 70, 94]
[0, 24, 48, 72]


In [34]:
import math

In [35]:
expectedC4z = np.array([-1+1j, 1+1j, -1+1j, 1+1j, 1-1j, 1+1j, 1-1j, -1+1j, 1+1j,
                  1-1j, -1-1j, 1-1j, -1-1j, 1+1j, 1-1j, -1-1j, -1+1j, 1+1j, 1-1j, -1-1j])/math.sqrt(2)

expectedIs = np.array([1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1])

expectedC4y = np.array( 
[[1,-1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0],
[3,-1,0,0,-1,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0],
[0,0,1,-1,0,-1,1,0,0,0,0,1,-1,0,0,0,0,0,0,0],
[0,0,1,1,0,-1,-1,0,0,0,0,1,1,0,0,0,0,0,0,0],
[3,1,0,0,-1,0,0,0,0,0,-3,0,0,0,0,0,0,0,0,0],
[0,0,2,-2,0,0,0,0,0,0,0,-2,2,0,0,0,0,0,0,0],
[0,0,2,2,0,0,0,0,0,0,0,-2,-2,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,1,-1,1,0,0,0,-1,1,-1,0,0,0,0],
[0,0,0,0,0,0,0,2,0,-2,0,0,0,-2,0,2,0,0,0,0],
[0,0,0,0,0,0,0,1,1,1,0,0,0,-1,-1,-1,0,0,0,0],
[1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
[0,0,1,-1,0,1,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0],
[0,0,1,1,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,1,-1,1,0,0,0,1,-1,1,0,0,0,0],
[0,0,0,0,0,0,0,2,0,-2,0,0,0,2,0,-2,0,0,0,0],
[0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,-1],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,-1,-1,3],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,-1,-3],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1]]
)/(2.0*math.sqrt(2.0))

In [36]:
np.allclose(np.diag(makeRepMat(o2h.elements[20], basis, allSpinsToBasis)), expectedC4y)

False

In [37]:
np.allclose(np.diag(makeRepMat(o2h.elements[22], basis, allSpinsToBasis)), expectedC4z)

True

In [38]:
(np.diag(makeRepMat(o2h.elements[22], basis, allSpinsToBasis))*math.sqrt(2)).round(4)

array([-1.+1.j,  1.+1.j, -1.+1.j,  1.+1.j,  1.-1.j,  1.+1.j,  1.-1.j,
       -1.+1.j,  1.+1.j,  1.-1.j, -1.-1.j,  1.-1.j, -1.-1.j,  1.+1.j,
        1.-1.j, -1.-1.j, -1.+1.j,  1.+1.j,  1.-1.j, -1.-1.j],
      dtype=complex64)

In [39]:
np.allclose(np.diag(makeRepMat(o2h.elements[48], basis, allSpinsToBasis)), expectedIs)

True

In [40]:
(makeRepMat(o2h.elements[20], basis, allSpinsToBasis)*math.sqrt(2.0)*2).round(4)

array([[ 1.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [-3.+0.j, -1.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  3.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  1.+0.j,  1.+0.j,  0.+0.j,  1.+0.j,  1.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  1.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  1.+0.j,  0.+0.j, -1.+0.j,  1.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j,  1.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 3.+0.j, -1.+0.j,  0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  3.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.

In [41]:
v=basis[0].spatial_rotate(o2h.elements[20])*2*math.sqrt(2)
print(v)

[ 1.+0.j -1.+0.j  0.+0.j  0.+0.j -1.+0.j  1.+0.j -0.+0.j -0.+0.j  0.+0.j
 -0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j -1.+0.j  1.+0.j
 -0.+0.j -0.+0.j  1.+0.j -1.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j -0.+0.j
 -0.+0.j -0.+0.j  0.+0.j -0.+0.j -0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j
 -0.+0.j  0.+0.j -0.+0.j -0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j  0.+0.j
 -0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j
 -0.+0.j -0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j
  0.+0.j]


In [42]:
fullVec_to_reduced(v, allSpinsToBasis, len(basis))*2*math.sqrt(2)

array([ 2.828427+0.j, -8.485281+0.j,  0.      +0.j,  0.      +0.j,
        8.485281+0.j,  0.      +0.j,  0.      +0.j,  0.      +0.j,
        0.      +0.j,  0.      +0.j, -2.828427+0.j,  0.      +0.j,
        0.      +0.j,  0.      +0.j,  0.      +0.j,  0.      +0.j,
        0.      +0.j,  0.      +0.j,  0.      +0.j,  0.      +0.j],
      dtype=complex64)

In [43]:
uQ(0).spatial_rotate(o2h.elements[20])

[(0.7071067811865476+0j), (-0.7071067811865476+0j), 0j, 0j]

In [44]:
np.eye(4)-np.matmul(gammas[3],gammas[1])

array([[ 1.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  1.+0.j, -1.+0.j],
       [ 0.+0.j,  0.+0.j,  1.+0.j,  1.+0.j]])

In [45]:
uQBar(0).spatial_rotate(o2h.elements[20])

[(0.7071067811865476+0j), (-0.7071067811865476+0j), 0j, 0j]

In [49]:
metric = np.matrix([[0. for j in range(len(rep[0]))] for i in range(len(rep[0]))], dtype='complex')
for g in rep:
  metric += np.matmul(np.matrix(g).getH(), np.matrix(g))
metric = (metric/len(o2h.elements)).round(8)

In [50]:
np.count_nonzero(metric - np.diag(np.diagonal(metric)))

0

In [51]:
(np.diagonal(metric)*3).round(4)

array([6.+0.j, 2.+0.j, 4.+0.j, 4.+0.j, 2.+0.j, 2.+0.j, 2.+0.j, 4.+0.j,
       2.+0.j, 4.+0.j, 6.+0.j, 4.+0.j, 4.+0.j, 4.+0.j, 2.+0.j, 4.+0.j,
       6.+0.j, 2.+0.j, 2.+0.j, 6.+0.j])

In [22]:
tot = 0
for irrep in o2h.elements[0].irreps:
  ops = operators(irrep, rep, o2h)
  tot += len(ops)*len(o2h.elements[0].irreps[irrep])
  print("{} ops in {}".format(len(ops), irrep))

print("{} operators across all irreps".format(tot))

0 ops in A1g
0 ops in A2u
0 ops in Eg
0 ops in T1u
0 ops in T2g
1 ops in G1g
0 ops in G2g
2 ops in Hg
0 ops in A1u
0 ops in A2g
0 ops in Eu
0 ops in T1g
0 ops in T2u
1 ops in G1u
0 ops in G2u
2 ops in Hu
20 operators across all irreps


In [23]:
for op in operators('G1g', rep, o2h):
  print_vec(op, basis)
  print()
print('----------------------------------')
for op in operators('G1g', rep, o2h, row=1):
  print_vec(op, basis)
  print()

(1+0j)*u_{a,0}*u_{a,2}*u_{a,3}+(-1+0j)*u_{a,1}*u_{a,2}*u_{a,2}+

----------------------------------
(1+0j)*u_{a,0}*u_{a,3}*u_{a,3}+(-1+0j)*u_{a,1}*u_{a,2}*u_{a,3}+



In [24]:
for op in operators('G1u', rep, o2h):
  print_vec(op, basis)
  print()
print('----------------------------------')
for op in operators('G1u', rep, o2h, row=1):
  print_vec(op, basis)
  print()

(1+0j)*u_{a,0}*u_{a,0}*u_{a,3}+(-1+0j)*u_{a,0}*u_{a,1}*u_{a,2}+

----------------------------------
(1+0j)*u_{a,0}*u_{a,1}*u_{a,3}+(-1+0j)*u_{a,1}*u_{a,1}*u_{a,2}+



In [25]:
for op in operators('Hg', rep, o2h):
  print_vec(op, basis)
  print()
print('----------------------------------')
for op in operators('Hg', rep, o2h, row=1):
  print_vec(op, basis)
  print()
print('----------------------------------')
for op in operators('Hg', rep, o2h, row=2):
  print_vec(op, basis)
  print()
print('----------------------------------')
for op in operators('Hg', rep, o2h, row=3):
  print_vec(op, basis)
  print()

(1+0j)*u_{a,0}*u_{a,0}*u_{a,0}+

(1+0j)*u_{a,0}*u_{a,2}*u_{a,2}+

----------------------------------
(1+0j)*u_{a,0}*u_{a,0}*u_{a,1}+

(1+0j)*u_{a,0}*u_{a,2}*u_{a,3}+(0.5001500150015001+0j)*u_{a,1}*u_{a,2}*u_{a,2}+

----------------------------------
(1+0j)*u_{a,0}*u_{a,1}*u_{a,1}+

(1+0j)*u_{a,0}*u_{a,3}*u_{a,3}+(1.999400119976005+0j)*u_{a,1}*u_{a,2}*u_{a,3}+

----------------------------------
(1+0j)*u_{a,1}*u_{a,1}*u_{a,1}+

(1+0j)*u_{a,1}*u_{a,3}*u_{a,3}+

